In [1]:
from z3 import *

from verification_oracle import verification_oracle


z3_x = Int('x')
z3_y = Int('y')

z3_input_variable_list = [z3_x, z3_y]
z3_function_declaration = max2 = Function('max2', IntSort(), IntSort(), IntSort())
z3_constraint = And(max2(z3_x, z3_y) >= z3_x, max2(z3_x, z3_y) >= z3_y, Or(max2(z3_y, z3_x) == z3_x, max2(z3_y, z3_x) == z3_y))

verification_oracle_instance = verification_oracle(z3_input_variable_list, z3_function_declaration, z3_constraint)
next(verification_oracle_instance)

In [2]:
from ordered_set import OrderedSet

from sympy.core.add import Add
from sympy.core.function import Function
from sympy.core.mul import Mul
from sympy.core.numbers import Integer
from sympy.core.relational import Equality, GreaterThan, LessThan
from sympy.core.symbol import Symbol
from sympy.logic.boolalg import And, Or, Not
from sympy.functions.elementary.piecewise import ExprCondPair, Piecewise


S = Symbol('S')
B = Symbol('B')
x = Symbol('x')
y = Symbol('y')

non_terminals = { S, B }
terminals = { x, y }
# non_terminals to a set of tuples containing the production_rule in reverse_polish_notation
production_rules = {
  S: {
    # x
    (x,),
    # y
    (y,),
    # 0
    (Integer(0),),
    # 1
    (Integer(1),),
    # + S S
    (S, S, (Add, 2)),
    # - S S
    (S, Integer(-1), S, (Mul, 2), (Add, 2)),
    # ite B S S
    (S, B, (ExprCondPair, 2), S, True, (ExprCondPair, 2), (Piecewise, 2)),
  },
  B: {
    # and B B
    (B, B, (And, 2)),
    # or B B
    (B, B, (Or, 2)),
    # not B
    (B, (Not, 1)),
    # <= S S
    (S, S, (LessThan, 2)),
    # = S S
    (S, S, (Equality, 2)),
    # >= S S
    (S, S, (GreaterThan, 2)),
  }
}
start_symbol = S

function_declaration = max2 = Function('max2')
constraint = And(GreaterThan(max2(x, y), x), GreaterThan(max2(x, y), y), Or(Equality(max2(y, x), x), Equality(max2(y, x), y)))

In [3]:
from sympy_expr_to_z3_expr_ref import sympy_expr_to_z3_expr_ref
from z3_expr_ref_to_sympy_expr import z3_expr_ref_to_sympy_expr

sympy_symbols_to_z3_expr_ref = {
    x: z3_x,
    y: z3_y
}

z3_expr_ref_to_sympy_symbols = {
    z3_x: x,
    z3_y: y
}

In [4]:
import itertools
from sys import stderr

import numpy as np
from sympy import *

from bottom_up_tree_search import bottom_up_enumeration
from evaluate_candidate_program_on_input import evaluate_candidate_program_on_input
from recursive_default_dict import RecursiveDefaultDict


def calculate_entropy(
        counterexample_set,
        term_to_covered_counterexample_set_dict
):
    if not len(counterexample_set):
        return 0.

    number_of_counterexamples_covered_by_terms_list = [
        len(covered_counterexample_set & counterexample_set)
        for covered_counterexample_set in term_to_covered_counterexample_set_dict.values()
    ]

    probabilities_of_counterexamples_labeled_by_terms_ndarray = np.zeros((
        len(counterexample_set), len(term_to_covered_counterexample_set_dict)
    ))
    for i, counterexample in enumerate(counterexample_set):
        # Fill row (with add one smoothing)
        original_row_sum = 0

        for j, (term, covered_counterexample_set) in enumerate(term_to_covered_counterexample_set_dict.items()):
            if counterexample in covered_counterexample_set:
                original_row_sum += number_of_counterexamples_covered_by_terms_list[j]
                probabilities_of_counterexamples_labeled_by_terms_ndarray[i, j] = \
                    number_of_counterexamples_covered_by_terms_list[j] + 1
            else:
                probabilities_of_counterexamples_labeled_by_terms_ndarray[i, j] = 1

        # Normalize row (with add one smoothing)
        probabilities_of_counterexamples_labeled_by_terms_ndarray[i, :] /= (
                original_row_sum + len(term_to_covered_counterexample_set_dict))

    # Calculate the (equally weighted) sum of all rows, and normalize
    probabilities_of_terms_labeling_any_counterexample = np.sum(
        probabilities_of_counterexamples_labeled_by_terms_ndarray, axis=0)
    probabilities_of_terms_labeling_any_counterexample /= np.sum(
        probabilities_of_terms_labeling_any_counterexample)

    # Calculate entropy
    entropy = -np.sum(probabilities_of_terms_labeling_any_counterexample * np.log2(
        probabilities_of_terms_labeling_any_counterexample))

    return entropy


def calculate_information_gain(
        predicate,
        counterexample_set,
        term_to_covered_counterexample_set_dict,
        predicate_to_covered_counterexample_set_dict
):
    if len(counterexample_set):
        covered_counterexample_set = predicate_to_covered_counterexample_set_dict[predicate]
        remaining_counterexample_set = counterexample_set - covered_counterexample_set

        return (len(covered_counterexample_set) * calculate_entropy(covered_counterexample_set, term_to_covered_counterexample_set_dict) + len(
            remaining_counterexample_set) * calculate_entropy(remaining_counterexample_set, term_to_covered_counterexample_set_dict)) / len(counterexample_set)
    else:
        return 0.


def divide_and_conquer_enumerative_approach(
        non_terminals,
        terminals,
        production_rules,
        term_non_terminal,
        predicate_non_terminal,
        function_declaration,
        constraint,
        indent_level=0
):
    indent = '    ' * indent_level

    term_iterator = bottom_up_enumeration(non_terminals, terminals, production_rules, term_non_terminal)
    predicate_iterator = bottom_up_enumeration(non_terminals, terminals, production_rules, predicate_non_terminal)

    # Updated from verification oracle
    counterexample_set = set()

    term_to_covered_counterexample_set_dict = dict()
    predicate_set = set()
    predicate_to_covered_counterexample_set_dict = dict()

    def generate_new_term():
        nonlocal function_declaration, constraint, term_iterator, term_to_covered_counterexample_set_dict, counterexample_set

        # Generate new term
        while True:
            generated_term_set, generated_term_size = next(term_iterator)
            if generated_term_set:
                break

        # Update term_to_covered_counterexample_set_dict
        for term in generated_term_set:
            covered_counterexample_set = set()
            for counterexample in counterexample_set:
                if evaluate_candidate_program_on_input(
                    function_declaration,
                    constraint,
                    term,
                    counterexample
                ):
                    covered_counterexample_set.add(counterexample)

            term_to_covered_counterexample_set_dict[term] = covered_counterexample_set

    def generate_new_predicate():
        nonlocal function_declaration, constraint, predicate_iterator, predicate_set, predicate_to_covered_counterexample_set_dict, counterexample_set

        # Generate new predicate (excluding True or False and generated ones)
        while True:
            generated_predicate_set, generated_predicate_size = next(predicate_iterator)
            filtered_generated_predicate_set = {
                predicate
                for predicate in generated_predicate_set
                if predicate not in (True, False) and predicate not in predicate_set and ~predicate not in predicate_set
            }
            
            if filtered_generated_predicate_set:
                break
            
        for predicate in filtered_generated_predicate_set:
            # Add to predicate_set
            predicate_set.add(predicate)

            # Update predicate_to_covered_counterexample_set_dict
            covered_counterexample_set = set()
            for counterexample in counterexample_set:
                if predicate.subs(counterexample):
                    covered_counterexample_set.add(counterexample)

            predicate_to_covered_counterexample_set_dict[predicate] = covered_counterexample_set


    def accept_counterexample(counterexample):
        nonlocal function_declaration, constraint, term_iterator, term_to_covered_counterexample_set_dict, counterexample_set

        counterexample_set.add(counterexample)

        for term, covered_counterexample_set in term_to_covered_counterexample_set_dict.items():
            if evaluate_candidate_program_on_input(
                function_declaration,
                constraint,
                term,
                counterexample
            ):
                covered_counterexample_set.add(counterexample)

        for predicate, covered_counterexample_set in predicate_to_covered_counterexample_set_dict.items():
            if predicate.subs(counterexample):
                covered_counterexample_set.add(counterexample)

    def construct_decision_tree(
        counterexample_set,
        term_to_covered_counterexample_set_dict,
        predicate_set,
        predicate_to_covered_counterexample_set_dict,
        indent_level=0
    ):
        indent = '    ' * indent_level
        print(indent, 'construct_decision_tree', counterexample_set, term_to_covered_counterexample_set_dict, predicate_set, predicate_to_covered_counterexample_set_dict, file=stderr)

        number_of_terms_covering_all_counterexamples = 0
        
        for term, covered_counterexample_set in term_to_covered_counterexample_set_dict.items():
            if counterexample_set.issubset(covered_counterexample_set):
                print(indent, term, 'covers all counterexamples, yielding', file=stderr)
                number_of_terms_covering_all_counterexamples += 1
                counterexample = yield term

        if number_of_terms_covering_all_counterexamples:
            return
        
        if set().union(*term_to_covered_counterexample_set_dict.values()) != counterexample_set:
            print(indent, 'set().union(*term_to_covered_counterexample_set_dict.values()) != input_set, nothing to yield', file=stderr)
            return
        
        filtered_predicates_set = {
            predicate for predicate, covered_counterexample_set in predicate_to_covered_counterexample_set_dict.items()
            if covered_counterexample_set and covered_counterexample_set != counterexample_set
        }

        if not filtered_predicates_set:
            print(indent, 'not filtered_predicates_set, nothing to yield', file=stderr)
            return
        
        remaining_predicates_set = filtered_predicates_set.copy()

        while remaining_predicates_set:
            # Calculating this every time tolerates counterexample_set, term_to_covered_counterexample_set_dict, predicate_to_covered_counterexample_set_dict CHANGING each time a counterexample is added
            predicate_to_information_gain_dict = {
                predicate: calculate_information_gain(
                    predicate,
                    counterexample_set,
                    term_to_covered_counterexample_set_dict,
                    predicate_to_covered_counterexample_set_dict
                )
                for predicate in remaining_predicates_set
            }
            
            print(indent, 'predicate_to_information_gain_dict:', predicate_to_information_gain_dict, file=stderr)
            
            predicate = min(
                predicate_to_information_gain_dict.keys(),
                key=lambda predicate: predicate_to_information_gain_dict[predicate]
            )
            
            remaining_predicates_set.remove(predicate)

            print(indent, 'predicate:', predicate, file=stderr)

            left_subtree_counterexample_set = predicate_to_covered_counterexample_set_dict[predicate].copy()

            left_subtree_term_to_covered_counterexample_set_dict = {
                term: left_subtree_counterexample_set & covered_counterexample_set
                for term, covered_counterexample_set
                in term_to_covered_counterexample_set_dict.items()
            }

            left_subtree_predicate_set = filtered_predicates_set - {predicate}

            left_subtree_predicate_to_covered_counterexample_set_dict = {
                left_subtree_predicate: left_subtree_counterexample_set & predicate_to_covered_counterexample_set_dict[left_subtree_predicate]
                for left_subtree_predicate in left_subtree_predicate_set
            }

            print(indent, 'left_subtree_counterexample_set:', left_subtree_counterexample_set, file=stderr)
            print(indent, 'left_subtree_term_to_covered_counterexample_set_dict:', left_subtree_term_to_covered_counterexample_set_dict, file=stderr)
            print(indent, 'left_subtree_predicate_set:', left_subtree_predicate_set, file=stderr)
            print(indent, 'left_subtree_predicate_to_covered_counterexample_set_dict:', left_subtree_predicate_to_covered_counterexample_set_dict, file=stderr)

            left_subtree_set = set(construct_decision_tree(
                left_subtree_counterexample_set,
                left_subtree_term_to_covered_counterexample_set_dict,
                left_subtree_predicate_set,
                left_subtree_predicate_to_covered_counterexample_set_dict,
                indent_level + 1
            ))

            if not left_subtree_set:
                print(indent, 'left_subtree_set is EMPTY, skipping this predicate', file=stderr)
                continue

            right_subtree_counterexample_set = counterexample_set - predicate_to_covered_counterexample_set_dict[predicate]

            right_subtree_term_to_covered_counterexample_set_dict = {
                term: right_subtree_counterexample_set & covered_counterexample_set_dict
                for term, covered_counterexample_set_dict
                in term_to_covered_counterexample_set_dict.items()
            }

            right_subtree_predicate_set = filtered_predicates_set - {predicate}

            right_subtree_predicate_to_covered_counterexample_set_dict = {
                right_subtree_predicate: right_subtree_counterexample_set & predicate_to_covered_counterexample_set_dict[right_subtree_predicate]
                for right_subtree_predicate in right_subtree_predicate_set
            }

            print(indent, 'right_subtree_counterexample_set:', right_subtree_counterexample_set, file=stderr)
            print(indent, 'right_subtree_term_to_covered_counterexample_set_dict:', right_subtree_term_to_covered_counterexample_set_dict, file=stderr)
            print(indent, 'right_subtree_predicate_set:', right_subtree_predicate_set, file=stderr)
            print(indent, 'right_subtree_predicate_to_covered_counterexample_set_dict:', right_subtree_predicate_to_covered_counterexample_set_dict, file=stderr)

            right_subtree_set = set(construct_decision_tree(
                right_subtree_counterexample_set,
                right_subtree_term_to_covered_counterexample_set_dict,
                right_subtree_predicate_set,
                right_subtree_predicate_to_covered_counterexample_set_dict,
                indent_level + 1
            ))

            if not right_subtree_set:
                print(indent, 'right_subtree_set is EMPTY, skipping this predicate', file=stderr)
                continue

            for left_subtree, right_subtree in itertools.product(left_subtree_set, right_subtree_set):
                decision_tree = Piecewise((left_subtree, predicate), (right_subtree, True))
                print(indent, 'yielding', decision_tree, file=stderr)
                yield decision_tree

    generate_new_term()
    while set().union(*term_to_covered_counterexample_set_dict.values()) != counterexample_set:
        generate_new_term()
    
    generate_new_predicate()

    while True:
        print(indent, 'counterexample_set:', counterexample_set, file=stderr)
        print(indent, 'term_to_covered_counterexample_set_dict:', term_to_covered_counterexample_set_dict, file=stderr)
        print(indent, 'predicate_to_covered_counterexample_set_dict:', predicate_to_covered_counterexample_set_dict, file=stderr)

        for decision_tree in construct_decision_tree(
            counterexample_set.copy(),
            term_to_covered_counterexample_set_dict.copy(),
            predicate_set.copy(),
            predicate_to_covered_counterexample_set_dict.copy(),
            indent_level + 1
        ):
            print(indent, 'decision_tree:', decision_tree, file=stderr)
            counterexample = yield decision_tree
            if counterexample is not None:
                print(indent, 'counterexample:', counterexample, file=stderr)
                accept_counterexample(counterexample)
                break
        else:
            print(indent, 'failed to construct decision_tree', file=stderr)
            
            generate_new_term()
            generate_new_predicate()
            
            continue

In [5]:
it = divide_and_conquer_enumerative_approach(
    non_terminals,
    terminals,
    production_rules,
    start_symbol,
    B,
    function_declaration,
    constraint
)

In [6]:
sympy_expr = next(it)

 counterexample_set: set()
 term_to_covered_counterexample_set_dict: {y: set(), 1: set(), x: set(), 0: set()}
 predicate_to_covered_counterexample_set_dict: {y <= 1: set(), x >= 1: set(), Eq(x, 0): set(), x <= 0: set(), y >= 1: set(), y >= 0: set(), y <= 0: set(), x >= 0: set(), Eq(y, 1): set(), Eq(x, y): set(), Eq(x, 1): set(), Eq(y, 0): set(), x <= y: set(), x >= y: set(), x <= 1: set()}
     construct_decision_tree set() {y: set(), 1: set(), x: set(), 0: set()} {y <= 1, x >= 1, Eq(x, 0), x <= 0, y >= 0, x >= y, y <= 0, x >= 0, Eq(y, 1), Eq(x, y), Eq(x, 1), Eq(y, 0), x <= y, y >= 1, x <= 1} {y <= 1: set(), x >= 1: set(), Eq(x, 0): set(), x <= 0: set(), y >= 1: set(), y >= 0: set(), y <= 0: set(), x >= 0: set(), Eq(y, 1): set(), Eq(x, y): set(), Eq(x, 1): set(), Eq(y, 0): set(), x <= y: set(), x >= y: set(), x <= 1: set()}
     y covers all counterexamples, yielding
 decision_tree: y


In [7]:
from frozendict import frozendict

In [8]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = it.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr y
z3_counterexample_input {x: 0, y: -1}


 counterexample: frozendict.frozendict({x: 0, y: -1})
 counterexample_set: {frozendict.frozendict({x: 0, y: -1})}
 term_to_covered_counterexample_set_dict: {y: set(), 1: set(), x: {frozendict.frozendict({x: 0, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1})}}
 predicate_to_covered_counterexample_set_dict: {y <= 1: {frozendict.frozendict({x: 0, y: -1})}, x >= 1: set(), Eq(x, 0): {frozendict.frozendict({x: 0, y: -1})}, x <= 0: {frozendict.frozendict({x: 0, y: -1})}, y >= 1: set(), y >= 0: set(), y <= 0: {frozendict.frozendict({x: 0, y: -1})}, x >= 0: {frozendict.frozendict({x: 0, y: -1})}, Eq(y, 1): set(), Eq(x, y): set(), Eq(x, 1): set(), Eq(y, 0): set(), x <= y: set(), x >= y: {frozendict.frozendict({x: 0, y: -1})}, x <= 1: {frozendict.frozendict({x: 0, y: -1})}}
     construct_decision_tree {frozendict.frozendict({x: 0, y: -1})} {y: set(), 1: set(), x: {frozendict.frozendict({x: 0, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1})}} {y <= 1, x >= 1, Eq(x, 0), x <= 0, y >= 0, x >= 

In [9]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = it.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr x
z3_counterexample_input {x: -1, y: 0}


 counterexample: frozendict.frozendict({x: -1, y: 0})
 counterexample_set: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}
 term_to_covered_counterexample_set_dict: {y: {frozendict.frozendict({x: -1, y: 0})}, 1: set(), x: {frozendict.frozendict({x: 0, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}}
 predicate_to_covered_counterexample_set_dict: {y <= 1: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, x >= 1: set(), Eq(x, 0): {frozendict.frozendict({x: 0, y: -1})}, x <= 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, y >= 1: set(), y >= 0: {frozendict.frozendict({x: -1, y: 0})}, y <= 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}, x >= 0: {frozendict.frozendict({x: 0, y: -1})}, Eq(y, 1): set(), Eq(x, y): set(), Eq(x, 1): set(), Eq(y, 0): {frozendict.frozendict({x: -1, y: 0})}, x <= y: {frozendict.frozendict({x: -1, y: 0})

In [10]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = it.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr 0
z3_counterexample_input {x: -1, y: -1}


 counterexample: frozendict.frozendict({x: -1, y: -1})
 counterexample_set: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}
 term_to_covered_counterexample_set_dict: {y: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, 1: set(), x: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}}
 predicate_to_covered_counterexample_set_dict: {y <= 1: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, x >= 1: set(), Eq(x, 0): {frozendict.frozendict({x: 0, y: -1})}, x <= 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, y >= 1: set(), y >= 0: {frozendict.frozendict({x: -1, y: 0})}, y <= 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}),

In [11]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = it.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr Piecewise((y, Eq(x, y)), (0, True))
z3_counterexample_input {x: 1, y: 0}


 counterexample: frozendict.frozendict({x: 1, y: 0})
 counterexample_set: {frozendict.frozendict({x: 1, y: 0}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}
 term_to_covered_counterexample_set_dict: {y: {frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, 1: {frozendict.frozendict({x: 1, y: 0})}, x: {frozendict.frozendict({x: 1, y: 0}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: -1})}, 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0})}}
 predicate_to_covered_counterexample_set_dict: {y <= 1: {frozendict.frozendict({x: 1, y: 0}), frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.frozendict({x: -1, y: -1})}, x >= 1: {frozendict.frozendict({x: 1, y: 0})}, Eq(x, 0): {frozendict.frozendict({x: 0, y: -1})}, x <= 0: {frozendict.frozendict({x: 0, y: -1}), frozendict.frozendict({x: -1, y: 0}), frozendict.fro

In [12]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = it.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr Piecewise((x, x >= y), (y, True))
z3_counterexample_input None


In [13]:
print(f'sympy_expr {sympy_expr}')

z3_expr_ref = sympy_expr_to_z3_expr_ref(sympy_symbols_to_z3_expr_ref, sympy_expr)
z3_counterexample_input = verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    counterexample_input = frozendict((
        (z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, key), z3_expr_ref_to_sympy_expr(z3_expr_ref_to_sympy_symbols, value))
        for key, value in z3_counterexample_input.items()
    ))

    sympy_expr = it.send(counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

sympy_expr Piecewise((x, x >= y), (y, True))
z3_counterexample_input None
